# 3.2 Creating LBD data from scratch

In this part, we will create a building model from scratch using the `BOT`, `BEO`, and `MEP` ontologies. This will result in an RDF graph or an ABox model (Assertion Box).

- `BOT`: https://w3id.org/bot#
- `BEO`: https://w3id.org/beo#
- `MEP`: https://pi.pauwel.be/voc/distributionelement#

## 1. Create initial Graph in RDFLib
We now setup again `rdflib` and our initial graph with the needed prefixes and namespaces.

In [29]:
# Import needed components from rdflib
from rdflib import Graph , Literal , BNode , Namespace , RDF , RDFS , OWL , URIRef

# initiate triple store, i.e. Graph()
g = Graph()

# Add namespaces and prefixes for ontologies
g.bind("owl", OWL)
BOT = Namespace("https://w3id.org/bot#")
g.bind("bot", BOT)
BEO = Namespace("https://w3id.org/beo#")
g.bind("beo", BEO)
MEP = Namespace("https://pi.pauwel.be/voc/distributionelement#")
g.bind("mep", MEP)

# Add namespace and prefix for instance graph (ABox)
INST = Namespace("https://example.org/OurBuilding#")
g.bind("", INST) # bind to default empty prefix
g.bind("inst", INST) # bind to inst prefix

# Initiate ontology entity
s = URIRef("https://example.org/OurBuilding")
p = RDF.type
o = OWL.Ontology
g.add((s, p, o))

<Graph identifier=N5fb5aaa27b3e4e87836d3924d55e7f64 (<class 'rdflib.graph.Graph'>)>

## 2. Adding sites, buildings, building storeys, and spaces
We will add in our graph:
- 1 site
- 1 building
- 3 storeys
- a different number of spaces per storey

Let's start with the site.

In [30]:
# Add site
s = INST["site_1"]
p = RDF.type
o = BOT["Site"]
g.add((s, p, o))
# Add a simple name
g.add((s, RDFS.label, Literal("Site 1")))

<Graph identifier=N5fb5aaa27b3e4e87836d3924d55e7f64 (<class 'rdflib.graph.Graph'>)>

Now let's add also the building and all storeys and spaces.

In [31]:
import random

# Add building
b = INST["building_1"]
p = RDF.type
o = BOT["Building"]
g.add((b, p, o))
g.add((b, RDFS.label, Literal("Our building")))

# Add building to site
g.add((s, BOT["hasBuilding"], b))

spacecounter = 0

# Add 3 storeys
for i in range(3):
    st = INST["storey_"+str(i)]
    p = RDF.type
    o = BOT["Storey"]
    g.add((st, p, o))
    g.add((st, RDFS.label, Literal("Storey " + str(i))))
    
    # Add storey to building
    g.add((b, BOT["hasStorey"], st))

    # Add random number of spaces to each storey
    number = random.randint(1, 10)
    for j in range(number):
        sp = INST["space_"+str(spacecounter)]
        p = RDF.type
        o = BOT["Space"]
        g.add((sp, p, o))
        g.add((sp, RDFS.label, Literal("Space " + str(spacecounter))))
        spacecounter+=1
        print(spacecounter)

        # Add space to storey
        g.add((st, BOT["hasSpace"], sp))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


## 3. Inspect our results
Let's see what we created.

In [23]:
for stmt in g:
    print(stmt)

print()
print("We have the following number of statements:")
print(len(g))

(rdflib.term.URIRef('https://example.org/OurBuilding#storey_1'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.Literal('Storey 1'))
(rdflib.term.URIRef('https://example.org/OurBuilding#storey_1'), rdflib.term.URIRef('https://w3id.org/bot#hasSpace'), rdflib.term.URIRef('https://example.org/OurBuilding#space_11'))
(rdflib.term.URIRef('https://example.org/OurBuilding#space_13'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.Literal('Space 13'))
(rdflib.term.URIRef('https://example.org/OurBuilding#site_1'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.Literal('Site 1'))
(rdflib.term.URIRef('https://example.org/OurBuilding#space_19'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.Literal('Space 19'))
(rdflib.term.URIRef('https://example.org/OurBuilding#space_9'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://w3i

How many spaces do we have?

In [24]:
# Exercise: Add your SPARQL query here to list all spaces. 

ourQuery = """PREFIX bot: <https://w3id.org/bot#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?s
WHERE{ ?s rdf:type bot:Space }"""

qres = g.query(ourQuery)

print( "Found these spaces: " )
for row in qres:
    print(f"{row.s}")

Found these spaces: 
https://example.org/OurBuilding#space_0
https://example.org/OurBuilding#space_1
https://example.org/OurBuilding#space_3
https://example.org/OurBuilding#space_5
https://example.org/OurBuilding#space_7
https://example.org/OurBuilding#space_9
https://example.org/OurBuilding#space_11
https://example.org/OurBuilding#space_13
https://example.org/OurBuilding#space_15
https://example.org/OurBuilding#space_17
https://example.org/OurBuilding#space_19


In which storeys are what spaces?

In [35]:
ourQuery = """PREFIX bot: <https://w3id.org/bot#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?s ?x ?name
WHERE{ 
    ?s rdf:type bot:Space .
    ?s rdfs:label ?name .
    ?x bot:hasSpace ?s
}"""

qres = g.query(ourQuery)

print( "Found these spaces: " )
for row in qres:
    print(f"{row.s} - name: {row.name} - on storey: {row.x}")

Found these spaces: 
https://example.org/OurBuilding#space_0 - name: Space 0 - on storey: https://example.org/OurBuilding#storey_0
https://example.org/OurBuilding#space_1 - name: Space 1 - on storey: https://example.org/OurBuilding#storey_0
https://example.org/OurBuilding#space_2 - name: Space 2 - on storey: https://example.org/OurBuilding#storey_0
https://example.org/OurBuilding#space_3 - name: Space 3 - on storey: https://example.org/OurBuilding#storey_0
https://example.org/OurBuilding#space_4 - name: Space 4 - on storey: https://example.org/OurBuilding#storey_0
https://example.org/OurBuilding#space_5 - name: Space 5 - on storey: https://example.org/OurBuilding#storey_0
https://example.org/OurBuilding#space_6 - name: Space 6 - on storey: https://example.org/OurBuilding#storey_0
https://example.org/OurBuilding#space_7 - name: Space 7 - on storey: https://example.org/OurBuilding#storey_0
https://example.org/OurBuilding#space_8 - name: Space 8 - on storey: https://example.org/OurBuildin

Now store the result in a file and examine the graph in [OntoText GraphDB](https://www.ontotext.com/products/graphdb/).

In [34]:
import os

g.serialize(destination = "output/OurHouse.ttl", format = "turtle")
print("Created output/OurHouse.ttl in folder:")
print(str(os.getcwd()))

Created OurHouse.ttl in folder:
c:\Users\20194060\Desktop\Git\pipauwel\SummerSchoolOfLDAC\Notebooks
